<a href="https://colab.research.google.com/github/josenalencar/ACLS_sim/blob/main/ACLS_SIMULATOR.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**O ACLS É UM CURSO SOBRE SEQUÊNCIAS**

E este é um simulador da sequência de atendimento em diversos casos clínicos diferentes.
Instruções de uso:
1. Aperte em play em um caso clínico de cada vez (o botão Play fica ao lado do botão "Mostrar Código").
2. Escreva a opção que você prefere dentre as demonstradas (a, b ou c?)

Não clique em mostrar código - vai poluir desnecessariamente a página. Caso algum problema ocorra, regarregue a página.

José Alencar (@josenalencar)

**CASO CLÍNICO 1. APERTE PLAY PARA JOGAR**

In [ ]:
# @title
from IPython.display import display, clear_output, HTML
import pandas as pd
import random

def get_user_choice(options):
    random.shuffle(options)
    options_str = " / ".join(f"{chr(96 + i+1)}. {option}" for i, option in enumerate(options))
    display(HTML(f"<strong><font size=4>Escolha uma opção: {options_str}</font></strong>"))
    choice = input().strip().lower()
    return options[ord(choice) - ord('a')]

def ask_question(question, correct_answer, wrong_answers, table, row, column_name, correct_update, clear=True):
    if clear:
        clear_output(wait=True)
    display(table)
    display(HTML(f"<strong><font size=4>{question}</font></strong>"))
    options = [correct_answer] + wrong_answers
    answer = get_user_choice(options)
    if answer == correct_answer:
        display(HTML(f"<strong><font size=4>Correto!</font></strong>"))
        table.at[row, column_name] = correct_update
        return True
    else:
        display(HTML(f"<strong><font size=4>ESTAVA TUDO INDO BEM, MAS AGORA ACONTECEU UM ERRO GRAVE.</font></strong>"))
        display(HTML(f"<strong><font size=4>A resposta correta seria: {correct_answer}</font></strong>"))
        input("Pressione Enter e depois clique em PLAY para jogar de novo...")
        return False

table = pd.DataFrame(columns=['Ciclo', 'Ritmo', 'Choque', 'RCP', 'Coach', 'Drogas', 'Outros'])
game_over = False

for ciclo in range(1, 5):
    if game_over:
        break

    if ciclo != 1:
        display(HTML(f"<strong><font size=4>DOIS MINUTOS</font></strong>"))

    table.at[ciclo-1, 'Ciclo'] = ciclo

    if ciclo == 1 or ciclo == 2:
        table.at[ciclo-1, 'Ritmo'] = 'FV'
        ritmo_question = "O ritmo é FV, o que fazer?"
        if ciclo == 2:
            ritmo_question = "DOIS MINUTOS. O ritmo é FV, o que fazer?"
        if not ask_question(ritmo_question, "chocar", ["comprimir", "preparar epinefrina"], table, ciclo-1, 'Choque', 'Feito'):
            game_over = True
            continue

    elif ciclo == 3:
        ritmo_question = "DOIS MINUTOS. O ritmo é uma linha reta, o que fazer agora?"
        if ask_question(ritmo_question, "Checar Cabos, ganhos e derivações", ["Chocar", "Comprimir"], table, ciclo-1, 'Ritmo', 'LR'):
            if ask_question("O ritmo é uma atividade elétrica, o que fazer agora?", "checar pulso", ["Começar RCP", "Começar protocolo de RCE"], table, ciclo-1, 'Ritmo', 'LR -> AESP'):
                if not ask_question("Não tem pulso. O que fazer agora?", "Começar RCP", ["Começar protocolo de RCE", "Fazer Epinefrina"], table, ciclo-1, 'RCP', 'Sim'):
                    game_over = True
                    continue
    elif ciclo == 4:
        if ask_question("DOIS MINUTOS.O ritmo é uma atividade elétrica. O que fazer agora?", "Checar pulso", ["Iniciar protocolo de RCE", "Fazer epinefrina"], table, ciclo-1, 'Ritmo', 'AECP'):
            if ask_question("TEM PULSO. O que fazer agora?", "Iniciar protocolo de RCE", ["Chocar", "Comprimir"], table, ciclo-1, 'RCP', 'Parou'):
                display(HTML(f"<strong><font size=4>PARABÉNS, O PACIENTE VOLTOU</font></strong>"))
                input("Pressione Enter e depois clique em PLAY para jogar de novo...")
                break

    if ciclo != 3:
        if not ask_question("Depois do choque, o que fazer?", "começar a RCP", ["checar ritmo", "preparar epinefrina"], table, ciclo-1, 'RCP', 'Sim'):
            game_over = True
            continue
        coach_action = "Estabelecer o coach" if ciclo == 1 else "Checar com o coach"
        if not ask_question("A RCP FOI INICIADA. O que fazer agora?", coach_action, ["fazer epinefrina", "buscar causas"], table, ciclo-1, 'Coach', 'Sim'):
            game_over = True
            continue
        drugs_action = "preparar epinefrina" if ciclo == 1 else "Administrar epinefrina e preparar Amiodarona 300"
        if not ask_question("O COACH ESTÁ ATUANDO.O que fazer agora?", drugs_action, ["fazer epinefrina", "ver ritmo"], table, ciclo-1, 'Drogas', 'Epi preparada' if ciclo==1 else 'Epi administrada e Amio 300 prep'):
            game_over = True
            continue

    if ciclo == 3:
        if not ask_question("A RCP FOI INICIADA. O que fazer agora?", "Checar com o coach", ["fazer epinefrina", "buscar causas"], table, ciclo-1, 'Coach', 'Sim'):
            game_over = True
            continue
        if not ask_question("O COACH ESTÁ ATUANDO. O que fazer agora?", "Preparar epinefrina", ["Fazer amiodarona 300 mg", "Fazer epinefrina"], table, ciclo-1, 'Drogas', 'Epi preparada'):
            game_over = True
            continue
        if not ask_question("VOCÊ ESTÁ INDO BEM. O que ainda falta fazer?", "checar causas", ["parar RCP", "fazer Amiodarona"], table, ciclo-1, 'Outros', 'Causas'):
            game_over = True
            continue

    if ciclo == 4:
        display(HTML(f"<strong><font size=4>DOIS MINUTOS. O ritmo é uma atividade elétrica. O que fazer agora?</font></strong>"))
        if ask_question("O ritmo é uma atividade elétrica. O que fazer agora?", "Checar pulso", ["Iniciar protocolo de RCE", "Fazer epinefrina"], table, ciclo-1, 'Ritmo', 'AECP'):
            if ask_question("TEM PULSO. O que fazer agora?", "Iniciar protocolo de RCE", ["Chocar", "Comprimir"], table, ciclo-1, 'RCP', 'Parou'):
                display(HTML(f"<strong><font size=4>PARABÉNS, O PACIENTE VOLTOU</font></strong>"))
                input("Pressione Enter e depois clique em PLAY para jogar de novo...")
                break

# Exibir tabela final
clear_output(wait=True)
display(table)

,Ciclo,Ritmo,Choque,RCP,Coach,Drogas,Outros
0,1,FV,Feito,Sim,Sim,Epi preparada,NaN
1,2,FV,Feito,Sim,Sim,Epi administrada e Amio 300 prep,NaN
2,3,LR,NaN,NaN,NaN,NaN,NaN


KeyboardInterrupt: ignored

**CASO CLÍNICO 2. APERTE PLAY PARA JOGAR**

In [ ]:
# @title
from IPython.display import display, clear_output, HTML
import pandas as pd
import random

def get_user_choice(options):
    random.shuffle(options)
    options_str = " / ".join(f"{chr(96 + i+1)}. {option}" for i, option in enumerate(options))
    display(HTML(f"<strong><font size=4>Escolha uma opção: {options_str}</font></strong>"))
    choice = input().strip().lower()
    return options[ord(choice) - ord('a')]

def ask_question(question, correct_answer, wrong_answers, table, row, column_name, correct_update, clear=True):
    if clear:
        clear_output(wait=True)
    display(table)
    display(HTML(f"<strong><font size=4>{question}</font></strong>"))
    options = [correct_answer] + wrong_answers
    answer = get_user_choice(options)
    if answer == correct_answer:
        display(HTML(f"<strong><font size=4>Correto!</font></strong>"))
        table.at[row, column_name] = correct_update
        return True
    else:
        display(HTML(f"<strong><font size=4>ESTAVA TUDO INDO BEM, MAS AGORA ACONTECEU UM ERRO GRAVE.</font></strong>"))
        display(HTML(f"<strong><font size=4>A resposta correta seria: {correct_answer}</font></strong>"))
        input("Pressione Enter e depois clique em PLAY para jogar de novo...")
        return False

table = pd.DataFrame(columns=['Ciclo', 'Ritmo', 'Choque', 'RCP', 'Coach', 'Drogas', 'Outros'])
game_over = False

for ciclo in range(1, 5):
    if game_over:
        break

    if ciclo != 1:
        display(HTML(f"<strong><font size=4>DOIS MINUTOS</font></strong>"))

    table.at[ciclo-1, 'Ciclo'] = ciclo

    # Ciclo 1
    if ciclo == 1:
            table.at[ciclo-1, 'Ritmo'] = 'TV'
            if not ask_question("O ritmo é TV. O que fazer?", "Checar pulso", ["Desfibrilar com 200 J", "Cardioverter com 100 J"], table, ciclo-1, 'Ritmo', 'TVSP'):
                game_over = True
                continue
            if not ask_question("O RITMO É TVSP. O que fazer agora?", "Desfibrilar com 200 J", ["Cardioverter sincronizado com 100 J", "Comprimir"], table, ciclo-1, 'Choque', 'Feito'):
                game_over = True
                continue
            if not ask_question("Depois do choque, o que fazer?", "começar a RCP", ["checar ritmo", "preparar epinefrina"], table, ciclo-1, 'RCP', 'Sim'):
                game_over = True
                continue
            if not ask_question("RCP INICIADA. O que fazer agora?", "Estabelecer o Coach", ["Checar ritmo", "administrar epinefrina"], table, ciclo-1, 'Coach', 'Sim'):
                game_over = True
                continue
            if not ask_question("COACH ATUANDO. O que fazer agora?", "Preparar epinefrina", ["Checar ritmo", "administrar epinefrina"], table, ciclo-1, 'Drogas', 'Epi preparada'):
                game_over = True
                continue

    # Ciclo 2
    elif ciclo == 2:
        table.at[ciclo-1, 'Ritmo'] = 'FV'
        if not ask_question("DOIS MINUTOS. O ritmo agora é FV. O que fazer agora?", "Chocar", ["administrar epinefrina", "comprimir"], table, ciclo-1, 'Ritmo', 'FV'):
            game_over = True
            continue
        if not ask_question("Depois do choque, o que fazer?", "começar a RCP", ["checar ritmo", "preparar epinefrina"], table, ciclo-1, 'RCP', 'Sim'):
            game_over = True
            continue
        if not ask_question("RCP INICIADA. O que fazer agora?", "Checar o Coach", ["Checar ritmo", "administrar epinefrina"], table, ciclo-1, 'Coach', 'Sim'):
            game_over = True
            continue
        if not ask_question("COACH ATUANDO. O que fazer agora?", "Administrar Epinefrina e Preparar 300 mg de Amiodarona", ["Administrar Amiodarona 300 mg", "checar ritmo"], table, ciclo-1, 'Drogas', 'Epi administrada e Amio 300 prep'):
            game_over = True
            continue

    # Ciclo 3
    elif ciclo == 3:
        table.at[ciclo-1, 'Ritmo'] = 'Atividade elétrica'
        if not ask_question("DOIS MINUTOS. O ritmo é uma atividade elétrica. O que fazer agora?", "checar pulso", ["Começar RCP", "Começar protocolo de RCE"], table, ciclo-1, 'Ritmo', 'AESP'):
            game_over = True
            continue
        if not ask_question("RCP INICIADA. O que fazer agora?", "Checar o Coach", ["Administrar amiodarona", "administrar epinefrina"], table, ciclo-1, 'Coach', 'Sim'):
            game_over = True
            continue
        if not ask_question("COACH ATUANDO. O que fazer agora?", "Preparar Epinefrina", ["Administrar Amiodarona 300 mg", "checar ritmo"], table, ciclo-1, 'Drogas', 'Epi preparada'):
            game_over = True
            continue
        if not ask_question("VOCÊ ESTÁ INDO BEM. O que ainda falta fazer?", "Checar causas", ["Parar RCP", "Fazer Amiodarona"], table, ciclo-1, 'Outros', 'Causas'):
            game_over = True
            continue

    elif ciclo == 4:
        if ask_question("DOIS MINUTOS.O ritmo é uma atividade elétrica. O que fazer agora?", "Checar pulso", ["Iniciar protocolo de RCE", "Fazer epinefrina"], table, ciclo-1, 'Ritmo', 'AECP'):
            if ask_question("TEM PULSO. O que fazer agora?", "Iniciar protocolo de RCE", ["Chocar", "Comprimir"], table, ciclo-1, 'RCP', 'Parou'):
                display(HTML(f"<strong><font size=4>PARABÉNS, O PACIENTE VOLTOU</font></strong>"))
                input("Pressione Enter e depois clique em PLAY para jogar de novo...")
                break



# Exibir tabela final
clear_output(wait=True)
display(table)

**CASO CLÍNICO 3: APERTE PLAY PARA JOGAR**

In [ ]:
# @title
from IPython.display import display, clear_output, HTML
import pandas as pd
import random

def get_user_choice(options):
    random.shuffle(options)
    options_str = " / ".join(f"{chr(96 + i+1)}. {option}" for i, option in enumerate(options))
    display(HTML(f"<strong><font size=4>Escolha uma opção: {options_str}</font></strong>"))
    choice = input().strip().lower()
    return options[ord(choice) - ord('a')]

def ask_question(question, correct_answer, wrong_answers, table, row, column_name, correct_update, clear=True):
    if clear:
        clear_output(wait=True)
    display(table)
    display(HTML(f"<strong><font size=4>{question}</font></strong>"))
    options = [correct_answer] + wrong_answers
    answer = get_user_choice(options)
    if answer == correct_answer:
        display(HTML(f"<strong><font size=4>Correto!</font></strong>"))
        table.at[row, column_name] = correct_update
        return True
    else:
        display(HTML(f"<strong><font size=4>ESTAVA TUDO INDO BEM, MAS AGORA ACONTECEU UM ERRO GRAVE.</font></strong>"))
        display(HTML(f"<strong><font size=4>A resposta correta seria: {correct_answer}</font></strong>"))
        input("Pressione Enter e depois clique em PLAY para jogar de novo...")
        return False

table = pd.DataFrame(columns=['Ciclo', 'Ritmo', 'Choque', 'RCP', 'Coach', 'Drogas', 'Outros'])
game_over = False

for ciclo in range(1, 6):
    if game_over:
        break

    if ciclo != 1:
        display(HTML(f"<strong><font size=4>DOIS MINUTOS</font></strong>"))

    table.at[ciclo-1, 'Ciclo'] = ciclo

    # Ciclo 1
    if ciclo == 1:
            table.at[ciclo-1, 'Ritmo'] = 'FV'
            if not ask_question("O RITMO É FV. O que fazer agora?", "Desfibrilar com 200 J", ["Cardioverter sincronizado com 100 J", "Comprimir"], table, ciclo-1, 'Choque', 'Feito'):
                game_over = True
                continue
            if not ask_question("Depois do choque, o que fazer?", "começar a RCP", ["checar ritmo", "preparar epinefrina"], table, ciclo-1, 'RCP', 'Sim'):
                game_over = True
                continue
            if not ask_question("RCP INICIADA. O que fazer agora?", "Estabelecer o Coach", ["Checar ritmo", "administrar epinefrina"], table, ciclo-1, 'Coach', 'Sim'):
                game_over = True
                continue
            if not ask_question("COACH ATUANDO. O que fazer agora?", "Preparar epinefrina", ["Checar ritmo", "administrar epinefrina"], table, ciclo-1, 'Drogas', 'Epi preparada'):
                game_over = True
                continue

    # Ciclo 2
    elif ciclo == 2:
        table.at[ciclo-1, 'Ritmo'] = 'FV'
        if not ask_question("DOIS MINUTOS. O ritmo agora é FV. O que fazer agora?", "Chocar", ["administrar epinefrina", "comprimir"], table, ciclo-1, 'Choque', 'Feito'):
            game_over = True
            continue
        if not ask_question("Depois do choque, o que fazer?", "começar a RCP", ["checar ritmo", "preparar epinefrina"], table, ciclo-1, 'RCP', 'Sim'):
            game_over = True
            continue
        if not ask_question("RCP INICIADA. O que fazer agora?", "Checar o Coach", ["Checar ritmo", "administrar epinefrina"], table, ciclo-1, 'Coach', 'Sim'):
            game_over = True
            continue
        if not ask_question("COACH ATUANDO. O que fazer agora?", "Administrar Epinefrina e Preparar 300 mg de Amiodarona", ["Administrar Amiodarona 300 mg", "checar ritmo"], table, ciclo-1, 'Drogas', 'Epi administrada e Amio 300 prep'):
            game_over = True
            continue

    # Ciclo 3
    elif ciclo == 3:
        table.at[ciclo-1, 'Ritmo'] = 'FV'
        if not ask_question("DOIS MINUTOS. O ritmo ainda é FV. O que fazer agora?", "Chocar", ["administrar epinefrina", "comprimir"], table, ciclo-1, 'Choque', 'Feito'):
            game_over = True
            continue
        if not ask_question("Depois do choque, o que fazer?", "começar a RCP", ["checar ritmo", "preparar epinefrina"], table, ciclo-1, 'RCP', 'Sim'):
            game_over = True
            continue
        if not ask_question("RCP INICIADA. O que fazer agora?", "Checar o Coach", ["Checar ritmo", "administrar epinefrina"], table, ciclo-1, 'Coach', 'Sim'):
            game_over = True
            continue
        if not ask_question("COACH ATUANDO. O que fazer agora?", "Administrar Amiodarona 300 mg e Preparar Epinefrina", ["Administrar Atropina 1 mg", "checar ritmo"], table, ciclo-1, 'Drogas', 'Amio 300 administrada e Epi preparada'):
            game_over = True
            continue

    # Ciclo 4
    elif ciclo == 4:
        table.at[ciclo-1, 'Ritmo'] = 'LR'
        if not ask_question("DOIS MINUTOS. O ritmo é uma linha reta. O que fazer agora?", "Checar cabos, ganhos e derivações", ["Começar RCP", "Administrar epinefrina"], table, ciclo-1, 'Ritmo', 'LR'):
            game_over = True
            continue
        if not ask_question("CHECADO TUDO e o ritmo virou uma atividade elétrica. O que fazer agora?", "checar pulso", ["Começar RCP", "Começar protocolo de RCE"], table, ciclo-1, 'Ritmo', 'LR -> AESP'):
            game_over = True
            continue
        if not ask_question("NÃO TEM PULSO. O que fazer agora?", "Começar RCP", ["Administrar Epinefrina", "Administrar Amiodarona"], table, ciclo-1, 'RCP', 'Sim'):
            game_over = True
            continue
        if not ask_question("RCP INICIADA. O que fazer agora?", "Checar o Coach", ["Administrar amiodarona", "administrar epinefrina"], table, ciclo-1, 'Coach', 'Sim'):
            game_over = True
            continue
        if not ask_question("COACH ATUANDO. O que fazer agora?", "Administrar Epinefrina", ["Administrar Amiodarona 300 mg", "checar ritmo"], table, ciclo-1, 'Drogas', 'Epi administrada'):
            game_over = True
            continue
        if not ask_question("VOCÊ ESTÁ INDO BEM. O que ainda falta fazer?", "Checar causas", ["Parar RCP", "Fazer Amiodarona"], table, ciclo-1, 'Outros', 'Causas'):
            game_over = True
            continue

    elif ciclo == 5:
        if ask_question("DOIS MINUTOS.O ritmo é uma atividade elétrica. O que fazer agora?", "Checar pulso", ["Iniciar protocolo de RCE", "Fazer epinefrina"], table, ciclo-1, 'Ritmo', 'AECP'):
            if ask_question("TEM PULSO. O que fazer agora?", "Iniciar protocolo de RCE", ["Chocar", "Comprimir"], table, ciclo-1, 'RCP', 'Parou'):
                display(HTML(f"<strong><font size=4>PARABÉNS, O PACIENTE VOLTOU</font></strong>"))
                input("Pressione Enter e depois clique em PLAY para jogar de novo...")
                break


# Exibir tabela final
clear_output(wait=True)
display(table)

**CASO CLÍNICO 4: APERTE PLAY PARA JOGAR**

In [ ]:
# @title
from IPython.display import display, clear_output, HTML
import pandas as pd
import random

def get_user_choice(options):
    random.shuffle(options)
    options_str = " / ".join(f"{chr(96 + i+1)}. {option}" for i, option in enumerate(options))
    display(HTML(f"<strong><font size=4>Escolha uma opção: {options_str}</font></strong>"))
    choice = input().strip().lower()
    return options[ord(choice) - ord('a')]

def ask_question(question, correct_answer, wrong_answers, table, row, column_name, correct_update, clear=True):
    if clear:
        clear_output(wait=True)
    display(table)
    display(HTML(f"<strong><font size=4>{question}</font></strong>"))
    options = [correct_answer] + wrong_answers
    answer = get_user_choice(options)
    if answer == correct_answer:
        display(HTML(f"<strong><font size=4>Correto!</font></strong>"))
        table.at[row, column_name] = correct_update
        return True
    else:
        display(HTML(f"<strong><font size=4>ESTAVA TUDO INDO BEM, MAS AGORA ACONTECEU UM ERRO GRAVE.</font></strong>"))
        display(HTML(f"<strong><font size=4>A resposta correta seria: {correct_answer}</font></strong>"))
        input("Pressione Enter e depois clique em PLAY para jogar de novo...")
        return False

table = pd.DataFrame(columns=['Ciclo', 'Ritmo', 'Choque', 'RCP', 'Coach', 'Drogas', 'Outros'])
game_over = False

for ciclo in range(1, 4):
    if game_over:
        break

    if ciclo != 1:
        display(HTML(f"<strong><font size=4>DOIS MINUTOS</font></strong>"))

    table.at[ciclo-1, 'Ciclo'] = ciclo

    # Ciclo 1
    if ciclo == 1:
        table.at[ciclo-1, 'Ritmo'] = 'AESP'
        if not ask_question("O ritmo é uma atividade elétrica sem pulso. O que fazer agora?", "começar a RCP", ["Administrar epinefrina", "Estabelecer o coach"], table, ciclo-1, 'RCP', 'Sim'):
            game_over = True
            continue
        if not ask_question("RCP INICIADA. O que fazer agora?", "Estabelecer o Coach", ["Administrar amiodarona", "Checar o ritmo"], table, ciclo-1, 'Coach', 'Sim'):
            game_over = True
            continue
        if not ask_question("COACH ATUANDO. O que fazer agora?", "Administrar Epinefrina", ["Administrar Amiodarona 300 mg", "checar ritmo"], table, ciclo-1, 'Drogas', 'Epi administrada'):
            game_over = True
            continue
        if not ask_question("VOCÊ ESTÁ INDO BEM. O que ainda falta fazer?", "Checar causas", ["Parar RCP", "Fazer Amiodarona"], table, ciclo-1, 'Outros', 'Causas'):
            game_over = True
            continue

    # Ciclo 2
    elif ciclo == 2:
        table.at[ciclo-1, 'Ritmo'] = 'AESP'
        if not ask_question("O ritmo é uma atividade elétrica. O que fazer agora?", "Checar pulso", ["Administrar epinefrina", "Começar a RCP"], table, ciclo-1, 'Ritmo', 'AESP'):
            game_over = True
            continue
        if not ask_question("NÃO TEM PULSO. O que fazer agora?", "Começar RCP", ["Administrar epinefrina", "Começar os cuidados pós parada"], table, ciclo-1, 'RCP', 'Sim'):
            game_over = True
            continue
        if not ask_question("RCP INICIADA. O que fazer agora?", "Checar o Coach", ["Administrar amiodarona", "administrar epinefrina"], table, ciclo-1, 'Coach', 'Sim'):
            game_over = True
            continue
        if not ask_question("COACH ATUANDO. O que fazer agora?", "Preparar Epinefrina", ["Administrar Amiodarona 300 mg", "Administrar Epinefrina"], table, ciclo-1, 'Drogas', 'Epi preparada'):
            game_over = True
            continue
        if not ask_question("VOCÊ ESTÁ INDO BEM. O que ainda falta fazer?", "Checar causas", ["Parar RCP", "Fazer Amiodarona"], table, ciclo-1, 'Outros', 'Causas'):
            game_over = True
            continue

    elif ciclo == 3:
        if ask_question("DOIS MINUTOS.O ritmo é uma atividade elétrica. O que fazer agora?", "Checar pulso", ["Iniciar protocolo de RCE", "Fazer epinefrina"], table, ciclo-1, 'Ritmo', 'AECP'):
            if ask_question("TEM PULSO. O que fazer agora?", "Iniciar protocolo de RCE", ["Chocar", "Comprimir"], table, ciclo-1, 'RCP', 'Parou'):
                display(HTML(f"<strong><font size=4>PARABÉNS, O PACIENTE VOLTOU</font></strong>"))
                input("Pressione Enter e depois clique em PLAY para jogar de novo...")
                break


# Exibir tabela final
clear_output(wait=True)
display(table)

,Ciclo,Ritmo,Choque,RCP,Coach,Drogas,Outros
0,1,AESP,NaN,Sim,Sim,Epi administrada,Causas
1,2,AESP,NaN,NaN,NaN,NaN,NaN
